In [14]:
import os
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import pandas as pd
from torchvision import models
import torch.nn as nn
from fastai.vision import Path
import torch
from torch.autograd import Variable

In [15]:
device=torch.device('cuda' if torch.cuda.is_available else 'cpu')
print(device)

cuda


In [16]:
def encode(a):
    onehot = [0]*ALL_CHAR_SET_LEN
    idx = ALL_CHAR_SET.index(a)
    onehot[idx] += 1
    return onehot

In [17]:
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
ALPHABET = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
ALL_CHAR_SET = NUMBER + ALPHABET
ALL_CHAR_SET_LEN = len(ALL_CHAR_SET)
MAX_CAPTCHA = 5

In [18]:
class Mydataset(Dataset):
    def __init__(self, path, is_train=True, transform=None):
        self.path = path
        if is_train: self.img = os.listdir(self.path)[:1000]
        else: self.img = os.listdir(self.path)[1001:]
        try: self.img.remove('3bnfnd.png')
        except: pass
        self.transform = transform
        
    def __getitem__(self, idx):
        img_path = self.img[idx]
        img = Image.open(self.path/img_path)
        img = img.convert('L')
        label = Path(self.path/img_path).name[:-4]
        label_oh = []
        for i in label:
            label_oh += encode(i)
        if self.transform is not None:
            img = self.transform(img)
        return img, np.array(label_oh), label
    
    def __len__(self):
        return len(self.img)

In [11]:
from zipfile import ZipFile
file_name = '/captcha.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [19]:
transform = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
])
train_ds = Mydataset(Path('/content/samples/samples'), transform=transform)
test_ds = Mydataset(Path('/content/samples/samples'), False, transform)
train_dl = DataLoader(train_ds, batch_size=64, num_workers=0)
test_dl = DataLoader(train_ds, batch_size=1, num_workers=0)

In [20]:
model = models.resnet18(pretrained=False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN*MAX_CAPTCHA, bias=True)
model.to(device)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [21]:
loss_func = nn.MultiLabelSoftMarginLoss()
optm = torch.optim.Adam(model.parameters(), lr=0.001)

In [28]:
hist={'epochloss':[],'trainacc':[],'testacc':[]}
minl=10000
modelname='captcharesnet'
for epoch in range(40):
    lossval=[]
    for step, i in enumerate(train_dl):
        img, label_oh, label = i
        img = Variable(img).cuda()
        label_oh = Variable(label_oh.float()).cuda()
        pred = model(img)
        loss = loss_func(pred, label_oh)
        lossval.append(loss.item())
        optm.zero_grad()
        loss.backward()
        optm.step()
        print('eopch:', epoch+1, 'step:', step+1, 'loss:', loss.item())
    curr_epoch_loss=np.array(lossval).mean()
    if curr_epoch_loss<minl:
      minl=curr_epoch_loss
      bestmodel=model.state_dict()
    curr_epoch_loss=np.array(lossval).mean()
    hist['epochloss'].append(curr_epoch_loss)
torch.save(bestmodel,'{0}_{1:0.4f}.pth'.format(modelname,minl))
        

eopch: 1 step: 1 loss: 0.07529532164335251
eopch: 1 step: 2 loss: 0.07121072709560394
eopch: 1 step: 3 loss: 0.0686471164226532
eopch: 1 step: 4 loss: 0.06932112574577332
eopch: 1 step: 5 loss: 0.06654561311006546
eopch: 1 step: 6 loss: 0.07080364227294922
eopch: 1 step: 7 loss: 0.0714176669716835
eopch: 1 step: 8 loss: 0.06543561816215515
eopch: 1 step: 9 loss: 0.068722203373909
eopch: 1 step: 10 loss: 0.06956450641155243
eopch: 1 step: 11 loss: 0.07022838294506073
eopch: 1 step: 12 loss: 0.0643436536192894
eopch: 1 step: 13 loss: 0.06548964232206345
eopch: 1 step: 14 loss: 0.06337521225214005
eopch: 1 step: 15 loss: 0.06812246888875961
eopch: 1 step: 16 loss: 0.05322299525141716
eopch: 2 step: 1 loss: 0.06528767198324203
eopch: 2 step: 2 loss: 0.059424594044685364
eopch: 2 step: 3 loss: 0.061092764139175415
eopch: 2 step: 4 loss: 0.06147482991218567
eopch: 2 step: 5 loss: 0.05585820972919464
eopch: 2 step: 6 loss: 0.06304396688938141
eopch: 2 step: 7 loss: 0.06333392858505249
eopch: 

In [33]:
mod=torch.load('/content/captcharesnet_0.0000.pth')
mod.eval();
for step, (img, label_oh, label) in enumerate(test_dl):
    img = Variable(img).cuda()
    pred = mod(img)
    c0 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c = '%s%s%s%s%s' % (c0, c1, c2, c3, c4)

    print('label:', label[0], 'pred:', c)

label: n4wwn pred: n4wwn
label: m5meg pred: m5meg
label: de7f8 pred: de7f8
label: f228n pred: f228n
label: dw6mn pred: dw6mn
label: 44ype pred: 44ype
label: emwpn pred: emwpn
label: y4n6m pred: y4n6m
label: w6yne pred: w6yne
label: d2n8x pred: d2n8x
label: 5np4m pred: 5np4m
label: nm248 pred: nm248
label: fp382 pred: fp382
label: bxxfc pred: bxxfc
label: 7e2y7 pred: 7e2y7
label: n4b4m pred: n4b4m
label: 3ygde pred: 3ygde
label: pn7pn pred: pn7pn
label: p7fyp pred: p7fyp
label: gcx6f pred: gcx6f
label: nfcb5 pred: nfcb5
label: 5mfff pred: 5mfff
label: x6pdb pred: x6pdb
label: 8bbw8 pred: 8bbw8
label: m75bf pred: m75bf
label: 57gnx pred: 57gnx
label: 2nx38 pred: 2nx38
label: x6b5m pred: x6b5m
label: x74b2 pred: x74b2
label: 87d4c pred: 87d4c
label: f6ne5 pred: f6ne5
label: c2fb7 pred: c2fb7
label: 5n728 pred: 5n728
label: bn5mw pred: bn5mw
label: 2356g pred: 2356g
label: nnn5p pred: nnn5p
label: mbp2y pred: mbp2y
label: 5nxnn pred: 5nxnn
label: pmd3w pred: pmd3w
label: 8w875 pred: 8w875


In [30]:
path = F"/content/captcharesnet_0.0000.pth"
bestmodel.load_state_dict(torch.load(path))
bestmodel=bestmodel.to(device)
bestmodel

AttributeError: ignored